## How can we predict survival on the Titanic?

to do list for self
- 1 apply fcc principles
- 2 apply [https://www.linkedin.com/pulse/what-i-learned-analyzing-famous-titanic-dateset-murilo-gustineli/]
- 3 apply [https://python.plainenglish.io/revitalizing-cyclistic-bike-share-program-an-in-depth-data-exploration-556b52512bf8] - diff dataset but still
- 4 apply others? [https://www.kaggle.com/code/startupsci/titanic-data-science-solutions]

### Guiding Questions
- Which features are correlated with survival?
- Can we accurately predict survival with a simple model?
- Which model performs best on this dataset?


### Dataset description
xyz


In [2]:
# import dataset and look at it
# describe the features

## Data Cleaning

- Inspect nulls
- Drop/recode columns
- Feature engineering (like 'is_alone', deck extraction, etc.)

In [ ]:
# clean as needed
# maybe remove outliers?

### EDA

- Visualize survival by class, sex, age, family, fare
- Show correlations (heatmap, groupby stats)
- Write observations inline

In [3]:
# visualizeeee

### Modeling

- 5 models: Logistic Regression, Decision Tree, Random Forest, KNN, Naive Bayes
- Optionally: SVM, Gradient Boosting, or Perceptron
- Compare accuracy, precision, recall, AUC
- Pick a best model + explain why

### 1) Logistic Regression model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# log reg code

### 2) Decision Tree model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# decision tree code

### 3) Random Forest

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# random forest code

### 4) KNN model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# KNN code

### 5) Naive Bayes model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [6]:
# Naive Bayes code

### 6) SVM model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# SVM code

### 7) Gradient Boosting model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# Gradient Boosting code

### 8) Perceptron model

- description high level w figs (read articles / watch vids)
- pros and cons in general and for this dataset

[writeup]

In [ ]:
# Perceptron code

### Final Evaluation

- Confusion matrix, F1, ROC curve
- Feature importance chart

In [5]:
# final eval

[write up too]

### Wrap-up
- Final thoughts, takeaways
- What you’d do next with more time/data
- References or inspiration sources

[writeup]